In [61]:
import requests as r
import json
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import AutoDateLocator

import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve

#### 爬蟲

In [ ]:
def webCrawler(start, end, stockNo):
    data_list = []

    month_list = pd.date_range(str(start), str(end), freq='MS').strftime("%Y%m%d").tolist()

    # 去遍歷所有月份資料，其中 stockNo=2330 表示的是台積電的股票代碼
    for month in month_list:
        url = "https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date="+ month + "&stockNo=" + stockNo
        res = r.get(url)
        if res.status_code == 200:  # 確保請求成功
            stock_json = res.json()
            if 'data' in stock_json:  # 確保有 'data' 欄位
                data_list.extend(stock_json['data'])  # 將每月的資料添加到 list 中
            else:
                print("沒有 data 欄位")
        else:
            print("爬取失敗")

    # 將資料轉換為 DataFrame
    df = pd.DataFrame(data_list)
    df.columns = ['date', 'volume', 'amount', 'open', 'high', 'low', 'close', 'price diff', '# of trades']
    
    for row in range(df.shape[0]):
        date2 = df.iloc[row,0].split('/')
        df.iloc[row, 0] = datetime(int(date2[0]) + 1911, int(date2[1]), int(date2[2])) # 把民國轉成西元

    df['date'] = pd.to_datetime(df['date']).dt.date # 移除時間部分，只提取日期

    df = df.drop(columns=['amount', 'open', 'price diff', '# of trades']) # 刪掉不用的欄位

    # 把字串轉成浮點數
    for col in [1, 2, 3, 4]:
        for row in range(df.shape[0]):
            df.iloc[row, col] = float(df.iloc[row,col].replace(',', ''))
    return df


#### 產生技術指標

In [63]:
#計算移動平均線 (MA)
def moving_average(data, window):
    return data.rolling(window=window).mean()

# 計算乖離率 (BIAS)
def bias(data, ma):
    return ((data - ma) / ma) * 100

# 計算 K 和 D 值
def stochastic_oscillator(data, high, low, n):
    lowest_low = low.rolling(window=n).min()
    highest_high = high.rolling(window=n).max()
    rsv = ((data - lowest_low) / (highest_high - lowest_low)) * 100
    k = rsv.ewm(alpha=1/3, adjust=False).mean()
    d = k.ewm(alpha=1/3, adjust=False).mean()
    return k, d

# 計算 RSI
def rsi(data, n):
    delta = data.diff()
    gain = delta.where(delta > 0, 0).ewm(span=n, adjust=False).mean()
    loss = -delta.where(delta < 0, 0).ewm(span=n, adjust=False).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

# 計算 MACD 和信號線
def macd(data, short_window=12, long_window=26, signal_window=9):
   # 計算 DI
    DI = (data['high'] + data['low'] + 2 * data['close']) / 4
    # 計算短期和長期 EMA
    EMA_12 = DI.ewm(span=short_window, adjust=False).mean()
    EMA_26 = DI.ewm(span=long_window, adjust=False).mean()
    # 計算 DIF
    DIF = EMA_12 - EMA_26
    # 計算 MACD
    MACD = DIF.ewm(span=signal_window, adjust=False).mean()
    return MACD

# 計算威廉指標 (WMS%R)
def williams_r(data, high, low, n):
    highest_high = high.rolling(window=n).max()
    lowest_low = low.rolling(window=n).min()
    return ((highest_high - data) / (highest_high - lowest_low)) * 100

def generate_indicators(stock):
    
    stock['10MA'] = moving_average(stock['close'], 10)
    stock['30MA'] = moving_average(stock['close'], 30)
    stock['10BIAS'] = bias(stock['close'], stock['10MA'])
    stock['30BIAS'] = bias(stock['close'], stock['30MA'])
    stock['K'], stock['D'] = stochastic_oscillator(stock['close'], stock['high'], stock['low'], 9)
    stock['RSI'] = rsi(stock['close'], 9)
    stock['MACD']= macd(stock)
    stock['WMS%R'] = williams_r(stock['close'], stock['high'], stock['low'], 14)
    stock = stock.dropna().reset_index(drop=True)
    return stock

#### PLR 特徵提取

In [64]:
# 定義節點類別
class Node:
    def __init__(self, x, y, start_idx, end_idx):
        self.x = x
        self.y = y
        self.start_idx = start_idx
        self.end_idx = end_idx
        self.left = None
        self.right = None

In [65]:
# 計算點到直線的垂直距離
def point_to_line_distance(x0, y0, x1, y1, x2, y2):
    numerator = abs((y2 - y1)*x0 - (x2 - x1)*y0 + x2*y1 - y2*x1)
    denominator = np.sqrt((y2 - y1)**2 + (x2 - x1)**2)
    return numerator / (denominator + 1e-6)  # 加上極小值防止除以零

In [66]:
# PLR 分段函數，遵循 BuildTree(S) 演算法
def BuildTree(x, y, start_idx, end_idx, threshold):
    # 建立當前節點
    node = Node(x, y, start_idx, end_idx)

    # 畫一條從起點到終點的直線
    x1, y1 = x[start_idx], y[start_idx]
    x2, y2 = x[end_idx], y[end_idx]

    # 計算最大距離及其索引
    max_distance = 0
    index_of_max = start_idx
    for i in range(start_idx + 1, end_idx):
        distance = point_to_line_distance(x[i], y[i], x1, y1, x2, y2)
        #print(i, x[i], y[i], distance)
        if distance > max_distance:
            max_distance = distance
            index_of_max = i

    # 判斷是否需要分割
    if max_distance > threshold:
        # 如果分割點的收盤價等於起點或終點的收盤價，則不分割
        if y[index_of_max] == y[start_idx] or y[index_of_max] == y[end_idx]:
            return node
        # 分割為左右兩個子節點
        node.left = BuildTree(x, y, start_idx, index_of_max, threshold)
        node.right = BuildTree(x, y, index_of_max, end_idx, threshold)
    return node

In [67]:
# 遍歷樹，提取分段
def get_segments(node, segments):
    if node.left is None and node.right is None: # 判斷是不是葉節點
        segments.append((node.start_idx, node.end_idx))
    else:
        if node.left:
            get_segments(node.left, segments)
        if node.right:
            get_segments(node.right, segments)

In [68]:
def PLR(stock, threshold):
    # 準備數據
    x = np.arange(len(stock))
    y = stock['close'].values

    # 建立 PLR 樹
    root = BuildTree(x, y, 0, len(x) - 1, threshold)

    # 獲取分段結果
    segments = []
    get_segments(root, segments)
    return segments

#### 提取特徵

In [69]:
def get_features(segments, stock, x, y, threshold, feature_num):
    # 提取特徵並構建特徵矩陣
    features = []
    labels = []

    previous_label = None  # 用於記錄前一個標籤

    for i in range(len(segments) - 1):
        start_idx, end_idx = segments[i]
        next_start_idx, next_end_idx = segments[i + 1]

        # 當前段的特徵
        x_segment = x[start_idx:end_idx + 1]
        y_segment = y[start_idx:end_idx + 1]
        volume_segment = stock['volume'].values[start_idx:end_idx + 1]

        # 計算斜率
        slope = (y_segment[-1] - y_segment[0]) / (x_segment[-1] - x_segment[0] + 1e-6)

        # 段長
        length = end_idx - start_idx

        # 中點價格
        mid_price = y_segment[len(y_segment) // 2]

        # 平均交易量
        avg_volume = np.mean(volume_segment)

        # 交易量變化
        volume_change = (volume_segment[-1] - volume_segment[0]) / (volume_segment[0] + 1e-6)

        # 技術指標
        ma_10_segment = stock['10MA'].values[start_idx:end_idx + 1]
        ma_30_segment = stock['30MA'].values[start_idx:end_idx + 1]
        rsi_segment = stock['RSI'].values[start_idx:end_idx + 1]
        macd_segment = stock['MACD'].values[start_idx:end_idx + 1]
        wmsr_segment = stock['WMS%R'].values[start_idx:end_idx + 1]
        bias_10_segment = stock['10BIAS'].values[start_idx:end_idx + 1]
        bias_30_segment = stock['30BIAS'].values[start_idx:end_idx + 1]
        k_segment = stock['K'].values[start_idx:end_idx + 1]
        d_segment = stock['D'].values[start_idx:end_idx + 1]

        # 技術指標特徵
        ma_10_mean = np.mean(ma_10_segment)
        ma_30_mean = np.mean(ma_30_segment)
        ma_10_std = np.std(ma_10_segment)
        ma_30_std = np.std(ma_30_segment)

        rsi_mean = np.mean(rsi_segment)
        macd_mean = np.mean(macd_segment)
        wmsr_mean = np.mean(wmsr_segment)

        bias_10_mean = np.mean(bias_10_segment)
        bias_10_std = np.std(bias_10_segment)
        bias_30_mean = np.mean(bias_30_segment)
        bias_30_std = np.std(bias_30_segment)

        k_mean = np.mean(k_segment)
        d_mean = np.mean(d_segment)

        # 標籤：預測線段價格是上升（1）還是下降（0）
        # [0, 37], [37, 38] 這兩個線段是比較 37 和 38 這兩個位置的股價變化
        next_y_segment = y[next_start_idx:next_end_idx + 1]
        price_change = next_y_segment[-1] - next_y_segment[0]

        label = -1

        if price_change > threshold:
            label = 1
        elif price_change < -threshold:
            label = 0
        elif previous_label is not None:  # 當標籤為 -1 時，標記為與上一個有效標籤相同
            label = previous_label

        if label in [0, 1]:  # 只把滿足條件的片段當作訓練資料
            if feature_num > 12:
                features.append([
                    slope, length, mid_price, avg_volume, volume_change,
                    ma_10_mean, ma_30_mean, ma_10_std, ma_30_std,
                    rsi_mean, macd_mean, wmsr_mean,
                    bias_10_mean, bias_10_std, bias_30_mean, bias_30_std, k_mean, d_mean  # 新增的技術指標
                ])
            else:
                features.append([
                    slope, length, mid_price, avg_volume, volume_change,
                    ma_10_mean, ma_30_mean, ma_10_std, ma_30_std,
                    rsi_mean, macd_mean, wmsr_mean,
                ])
            labels.append(label)

            # 更新 previous_label
            previous_label = label

    # 轉換為 NumPy 數組
    X = np.array(features)
    y_labels = np.array(labels)
    
    return X, y_labels


#### 模型結果

In [70]:
def create_lstm_input(X, y, time_steps):
    X_lstm = []
    y_lstm = []
    for i in range(len(X) - time_steps):
        X_lstm.append(X[i:i+time_steps])
        y_lstm.append(y[i+time_steps])
        
    return np.array(X_lstm), np.array(y_lstm)

In [71]:
def run_model(X, y_labels, model):
    X_test_lstm, y_test_lstm = create_lstm_input(X, y_labels, time_steps=2)
    y_pred_prob = model.predict(X_test_lstm)
    
    fpr, tpr, thresholds = roc_curve(y_test_lstm, y_pred_prob)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]

    y_pred = (y_pred_prob >= optimal_threshold).astype(int).flatten()

    return y_pred

#### 最終輸出畫面

In [72]:
import base64
import io

In [73]:
def calculate_stock_cost(buy_price, quantity):

    handling_fee = buy_price * quantity * 0.001425 * 2  # 手續費
    transaction_tax = buy_price * quantity * 0.003  # 交易稅
    total_cost = (buy_price * quantity) + handling_fee + transaction_tax
    return total_cost

def calculate_stock_profit(buy_price, sell_price, quantity):

    total_cost = calculate_stock_cost(buy_price, quantity)
    total_revenue = sell_price * quantity
    profit = total_revenue - total_cost
    return profit

In [74]:
def get_result(y_pred, segments, stock, y):
    total_profit = 0
    buy_price = None
    buy_points = []  # 買入點的時間和價格
    sell_points = []  # 賣出點的時間和價格
    start = 0
    time_steps = 2
    
    # 計算獲利並記錄買賣點
    for i in range(len(y_pred)):
        if y_pred[i] == 1 and buy_price is None:  # 買入
            if start == 0:
                start = segments[i + time_steps][1]
            buy_price = y[segments[i + time_steps][1]]
            buy_points.append((segments[i + time_steps][1], buy_price))

        elif y_pred[i] == 0 and buy_price is not None:  # 賣出
            sell_price = y[segments[i + time_steps][1]]
            sell_points.append((segments[i + time_steps][1], sell_price))

            # 計算單次交易的成本與獲利
            quantity = 100  # 假設買賣股數固定為 100 股
            profit = calculate_stock_profit(buy_price, sell_price, quantity)
            total_profit += profit
            #print(f"此次交易獲利: {profit:.2f} 元")

            # 重置買入價格
            buy_price = None

    # 提取買賣點數據
    buy_times, buy_prices = zip(*buy_points) if buy_points else ([], [])
    sell_times, sell_prices = zip(*sell_points) if sell_points else ([], [])

    date = stock.iloc[start:]['date'].reset_index(drop=True)
    y_range = y[start:]

    # 繪製價格折線圖和買賣點
    plt.figure(figsize=(12, 6))
    plt.plot(date, y_range, label='Original Price', color='blue', linewidth=3) # 繪製價格折線圖

    # 買入點
    buy_dates = [date.iloc[buy_time - start] for buy_time in buy_times]
    plt.scatter(buy_dates, buy_prices, color='green', label='buy', marker='^', s=100)

    # 賣出點
    sell_dates = [date.iloc[sell_time - start] for sell_time in sell_times]
    plt.scatter(sell_dates, sell_prices, color='red', label='Sell', marker='v', s=100)

    # 格式化 x 軸為日期
    plt.gca().xaxis.set_major_locator(AutoDateLocator())  # 自動調整日期間隔

    # 旋轉 x 軸標籤
    plt.xticks(rotation=45)

    # 添加圖例與標題
    plt.xlabel('time index')
    plt.ylabel('price')
    plt.legend()
    plt.grid(True)

    #plt.show()
    
    # 將圖像保存到內存並轉換為 Base64
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    image_base64 = base64.b64encode(buf.getvalue()).decode('utf-8')
    buf.close()
    plt.close()

    return image_base64, total_profit


#### 後端

In [ ]:
from flask import Flask, render_template, request, jsonify
from joblib import dump, load

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/get_stock_data', methods=['POST'])
def get_stock_data():
    stock_symbol = request.form['stock_symbol']
    start_date = request.form['start_date']
    end_date = request.form['end_date']

    try:
        # 爬取數據
        stock_data = webCrawler(start_date, end_date, stock_symbol)
        
        stock_data = generate_indicators(stock_data)
        thresholds = {"2330": 4, "3443": 5, "3034": 4.5, "2379": 3.5}
        feature_nums = {"2330": 12, "3443": 12, "3034": 12, "2379": 12}

        threshold = thresholds.get(stock_symbol)
        feature_num = feature_nums.get(stock_symbol)

        # PLR 特徵提取
        segments = PLR(stock_data, threshold)
        X, y_labels = get_features(segments, stock_data, np.arange(len(stock_data)), stock_data['close'].values, threshold, feature_num)
        
        model_paths = {
            "2330": "models/2330_model.h5",
            "3443": "models/3443_model.h5",
            "3034": "models/3034_model.h5",
            "2379": "models/2379_model.h5",
        }
        model_path = model_paths.get(stock_symbol)

        model = load_model(model_path)
        y_pred = run_model(X, y_labels, model)

        image_base64, total_profit = get_result(y_pred, segments, stock_data, stock_data['close'].values)

        return jsonify({'status': 'success', 'image': image_base64, 'profit': total_profit})
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)})

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
INFO:werkzeug:Press CTRL+C to quit


(368, 5)
76
75 75
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


127.0.0.1 - - [14/Jan/2025 12:24:45] "POST /get_stock_data HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Jan/2025 12:24:45] "POST /get_stock_data HTTP/1.1" 200 -


[0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 1 0 1 1 0 1 0 0 1 0 1 1 1 0 0 0 1 0 1 0 1 1
 0 1 1 1 1 1 0 0 1 1 1 0 1 0 1 0 0 0 0 1 0 1 0 0 0 1 1 0 0 1 1 0 0 0 1 0]


...

127.0.0.1 - - [14/Jan/2025 12:30:29] "POST /get_stock_data HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Jan/2025 12:30:29] "POST /get_stock_data HTTP/1.1" 200 -


[0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0 1 1
 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 1]


(243, 5)
38
37 37
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step


127.0.0.1 - - [14/Jan/2025 12:30:41] "POST /get_stock_data HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Jan/2025 12:30:41] "POST /get_stock_data HTTP/1.1" 200 -


[1 1 0 0 0 1 0 0 0 1 0 1 0 1 1 1 0 1 1 0 1 1 0 0 1 0 1 1 1 0 0 1 1 1 1]
